In [116]:
#import packages
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

import duckdb, sqlalchemy

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Research Questions



## Data Cleaning
In order to simplify the analysis process, we've added a column to each of the initial dataframes called "Subject" so that when we combine the dataframes, we can easily identify which data set and subject the information relates to.


In [117]:
#read in csvs
ithaca = pd.read_csv("Ithaca.csv")
others = pd.read_csv("AdjacentCities.csv")
west = pd.read_csv("Erie.csv")
#print(ithaca.head())
#print(west.head())
others = others.dropna(axis=0,subset=['STATION'])
#print(others.head())

C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_18428\1485939667.py:2: DtypeWarning: Columns (9,15,17,19,21,27,29,31,33,35,37,39,41,47) have mixed types. Specify dtype option on import or set low_memory=False.
  ithaca = pd.read_csv("Ithaca.csv")
C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_18428\1485939667.py:3: DtypeWarning: Columns (27,29,31,33,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  others = pd.read_csv("AdjacentCities.csv")
C:\Users\Margaret Quirk\AppData\Local\Temp\ipykernel_18428\1485939667.py:4: DtypeWarning: Columns (7,9,11,13,15,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81) have mixed types. Specify dtype option on import or set low_memory=False.
  west = pd.read_csv("Erie.csv")


In [118]:
#Add column to indicate location (relative to ithaca)
ithaca["Location"] = "Central"
west['Location'] = "West"
#print(others['NAME'].unique())
locations = []
for i in others['NAME']:
    if "WATERTOWN" in i:
        locations.append('North')
    elif "BLACK RIVER" in i:
        locations.append("North")
    elif "ESPY" in i:
        locations.append("South")
    elif "BLOOMSBURG" in i:
        locations.append("South")
    elif "COBLESKILL" in i:
        locations.append("East")
    else:
        locations.append('N/A')
others['Location'] = locations
print(others['Location'].unique())


['North' 'South' 'East']


In [119]:
#Check all columns match before concatenation
columns_to_keep = []

for i in ithaca.columns.tolist():
    if (i in others.columns.tolist()) & (i in west.columns.tolist()) :
        columns_to_keep.append(i)
ithaca_good = ithaca[columns_to_keep]
others_good = others[columns_to_keep]
west_good = west[columns_to_keep]
        
if (ithaca_good.columns.tolist() == others_good.columns.tolist()) & (ithaca_good.columns.tolist() == west_good.columns.tolist()):
    print("Columns match: proceed")
    final_df = pd.concat([ithaca_good,others_good,west_good])
else:
    print(ithaca.columns)
    print(others.columns)
    
#print(final_df)

Columns match: proceed


In [120]:
#Check for null values in columns
print(final_df.columns)
#If a column has only null values, drop the column
null_cols = []
for c in final_df.columns:
    if final_df[c].isnull().all():
        null_cols.append(c)
if len(null_cols) == 0:
    print("No null columns")
else:
    print('Null columns:', null_cols)
    final_df = final_df.drop(null_cols)

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'DAPR',
       'DAPR_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES',
       'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TMAX',
       'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'Location'],
      dtype='object')
No null columns


In [121]:
#Create binary columns to indicate if there was trace of precipitation for snow and rain
precip_binary = []
snow_binary = []
for p in final_df['PRCP_ATTRIBUTES']:
    #print(type(p))
    if (type(p) == str):
        if 'T' in p:
            precip_binary.append(1)
        else:
            precip_binary.append(0)
    else:
        precip_binary.append(0)
        
for s in final_df['SNOW_ATTRIBUTES']:
    if (type(s) == str):
        if 'T' in s:
            snow_binary.append(1)
        else:
            snow_binary.append(0)
    else:
        snow_binary.append(0)
    
final_df['Precip Trace'] = precip_binary
final_df['Snow Trace'] = snow_binary

In [122]:
#Drop attributes columns now that we have created the binary columns
final_df = final_df.drop(columns = ['PRCP_ATTRIBUTES', 'SNOW_ATTRIBUTES'])
#print(final_df.columns)

In [104]:
#Create 5 separate data frames based on location for individual analyses when needed
central = final_df[final_df['Location']=="Central"]
north = final_df[final_df['Location']=="North"]
south = final_df[final_df['Location']=="South"]
east = final_df[final_df['Location']=="East"]
west = final_df[final_df['Location']=="West"]

## Data Description



## Data Limitations

No student ids - can't link individual students between datasets

Family size column is vague (over or under 3 people)

Several columns are ranges from 1-5 with little detail on how students were placed within these categories. 

In the failures column, if a student has failed more than 3 classes in the past, the value listed is 4 regardless of how many they've failed

## Exploratory Data Analysis

In [ ]:
#side by side linear regression of consumption vs grades for math and portuguese sets
#histogram for grades/consumption/absences